<a href="https://colab.research.google.com/github/Ankur0191/NTPC/blob/main/model3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi uvicorn tensorflow pandas numpy scikit-learn matplotlib seaborn requests
!pip install nest-asyncio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.9 MB/s eta 0:00:00


In [ ]:
!pip uninstall -y patsy && pip install --no-cache-dir patsy seaborn


Found existing installation: patsy 1.0.1
Uninstalling patsy-1.0.1:
  Successfully uninstalled patsy-1.0.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.9/232.9 kB 40.6 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade --no-cache-dir patsy statsmodels seaborn


In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
import nest_asyncio
import uvicorn
from fastapi import FastAPI

def fetch_nasa_power_data(lat, lon, start_year, end_year, parameter):
    """Fetches weather data from NASA POWER API."""
    url = f"https://power.larc.nasa.gov/api/temporal/daily/point?parameters={parameter}&community=RE&longitude={lon}&latitude={lat}&start={start_year}0101&end={end_year}1231&format=JSON"
    response = requests.get(url)
    data = response.json()
    return pd.DataFrame.from_dict(data['properties']['parameter'][parameter], orient='index', columns=[parameter])



In [ ]:
# Define NTPC power plants with coordinates
ntpc_solar_plants = {
    "Bhadla Solar": (27.485071, 71.997756),
    "Jetsar Solar": (9.0128, 76.9144),
    "Bhensada Solar": (24.1, 78.8)
}

ntpc_wind_plants = {
    "NTPC Hybrid SECI V": (23.2064, 78.382),
    "Dayapar Wind Farm": (23.6348, 68.9115)
}

# Fetch and preprocess data
def prepare_data_for_training(plant_dict, parameter):
    all_data = []
    for plant, coords in plant_dict.items():
        print(f"Fetching data for {plant} ({coords})...")
        df = fetch_nasa_power_data(coords[0], coords[1], 2015, 2024, parameter)
        df.index = pd.to_datetime(df.index, format='%Y%m%d')
        df['Plant'] = plant
        all_data.append(df)
    return pd.concat(all_data)

solar_data = prepare_data_for_training(ntpc_solar_plants, "ALLSKY_SFC_SW_DWN")
wind_data = prepare_data_for_training(ntpc_wind_plants, "WS10M")


Fetching data for Bhadla Solar ((27.485071, 71.997756))...
Fetching data for Jetsar Solar ((9.0128, 76.9144))...
Fetching data for Bhensada Solar ((24.1, 78.8))...
Fetching data for NTPC Hybrid SECI V ((23.2064, 78.382))...
Fetching data for Dayapar Wind Farm ((23.6348, 68.9115))...


In [ ]:
def train_lstm_model(data, feature, model_name):
    """Trains an LSTM model and saves it to disk."""
    data = data[[feature]].copy()
    data = data.dropna()
    data = data.values

    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(data)

    X, y = [], []
    for i in range(30, len(data_scaled) - 1):
        X.append(data_scaled[i-30:i])
        y.append(data_scaled[i])

    X, y = np.array(X), np.array(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    model = Sequential([
        keras.Input(shape=(30, 1)),
        LSTM(50, activation='relu', return_sequences=True),
        Dropout(0.2),
        LSTM(50, activation='relu'),
        Dropout(0.2),
        Dense(1)
    ])

    model.compile(optimizer='adam', loss='mse')

    try:
        model = load_model(f"{model_name}.h5")
        print(f"Loaded existing model: {model_name}.h5")
    except:
        print(f"Training new model: {model_name}")
        model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))
        model.save(f"{model_name}.h5")

    return model, scaler

solar_model, solar_scaler = train_lstm_model(solar_data, "ALLSKY_SFC_SW_DWN", "solar_model")
wind_model, wind_scaler = train_lstm_model(wind_data, "WS10M", "wind_model")


Training new model: solar_model
Epoch 1/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 19s 28ms/step - loss: 0.1203 - val_loss: 0.0017
Epoch 2/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 19s 26ms/step - loss: 0.0110 - val_loss: 6.4673e-04
Epoch 3/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 21s 26ms/step - loss: 0.0094 - val_loss: 7.4554e-04
Epoch 4/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - loss: 0.0073 - val_loss: 5.1653e-04
Epoch 5/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 15s 27ms/step - loss: 0.0057 - val_loss: 4.6032e-04
Epoch 6/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - loss: 0.0052 - val_loss: 4.6883e-04
Epoch 7/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - loss: 0.0031 - val_loss: 4.5767e-04
Epoch 8/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 14s 26ms/step - loss: 0.0020 - val_loss: 4.7789e-04
Epoch 9/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 14s 26ms/step - loss: 0.0012 - val_loss: 5.2158e-04
Epoch 10/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - loss: 9.6005e-04 - val_loss: 4.6743e-04
Epoch 11/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 25s 33

Training new model: wind_model
Epoch 1/50
364/364 ━━━━━━━━━━━━━━━━━━━━ 14s 27ms/step - loss: 0.0158 - val_loss: 0.0109
Epoch 2/50
364/364 ━━━━━━━━━━━━━━━━━━━━ 10s 27ms/step - loss: 0.0083 - val_loss: 0.0103
Epoch 3/50
364/364 ━━━━━━━━━━━━━━━━━━━━ 10s 27ms/step - loss: 0.0072 - val_loss: 0.0098
Epoch 4/50
364/364 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - loss: 0.0067 - val_loss: 0.0082
Epoch 5/50
364/364 ━━━━━━━━━━━━━━━━━━━━ 20s 27ms/step - loss: 0.0060 - val_loss: 0.0070
Epoch 6/50
364/364 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - loss: 0.0058 - val_loss: 0.0065
Epoch 7/50
364/364 ━━━━━━━━━━━━━━━━━━━━ 10s 27ms/step - loss: 0.0054 - val_loss: 0.0060
Epoch 8/50
364/364 ━━━━━━━━━━━━━━━━━━━━ 10s 28ms/step - loss: 0.0052 - val_loss: 0.0057
Epoch 9/50
364/364 ━━━━━━━━━━━━━━━━━━━━ 10s 26ms/step - loss: 0.0053 - val_loss: 0.0056
Epoch 10/50
364/364 ━━━━━━━━━━━━━━━━━━━━ 10s 26ms/step - loss: 0.0047 - val_loss: 0.0062
Epoch 11/50
364/364 ━━━━━━━━━━━━━━━━━━━━ 10s 27ms/step - loss: 0.0049 - val_loss: 0.0055

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.losses import MeanSquaredError

def train_lstm_model(data, feature, model_name):
    """Loads an existing LSTM model if available, otherwise trains a new one."""
    data = data[[feature]].copy()
    data = data.dropna()
    data = data.values

    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(data)

    X, y = [], []
    for i in range(30, len(data_scaled) - 1):
        X.append(data_scaled[i-30:i])
        y.append(data_scaled[i])

    X, y = np.array(X), np.array(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # ✅ Check if model file exists before training
    if os.path.exists(f"{model_name}.h5"):
        print(f"✅ Loading existing model: {model_name}.h5")
        model = load_model(f"{model_name}.h5", custom_objects={"mse": MeanSquaredError()})  # ✅ Fix applied
    else:
        print(f"🚀 Training new model: {model_name}")
        model = Sequential([
            keras.Input(shape=(30, 1)),
            LSTM(50, activation='relu', return_sequences=True),
            Dropout(0.2),
            LSTM(50, activation='relu'),
            Dropout(0.2),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss=MeanSquaredError())  # ✅ Use explicit function
        model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))
        model.save(f"{model_name}.h5")  # ✅ Save model after training

    return model, scaler, X_test, y_test  # ✅ Returns X_test and y_test

# Train or Load Models
solar_model, solar_scaler, X_test_solar, y_test_solar = train_lstm_model(solar_data, "ALLSKY_SFC_SW_DWN", "solar_model")
wind_model, wind_scaler, X_test_wind, y_test_wind = train_lstm_model(wind_data, "WS10M", "wind_model")


✅ Loading existing model: solar_model.h5
✅ Loading existing model: wind_model.h5


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.losses import MeanSquaredError

def train_lstm_model(data, feature, model_name):
    """Loads an existing LSTM model if available, otherwise trains a new one."""
    data = data[[feature]].copy()
    data = data.dropna()
    data = data.values

    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(data)

    X, y = [], []
    for i in range(30, len(data_scaled) - 1):
        X.append(data_scaled[i-30:i])
        y.append(data_scaled[i])

    X, y = np.array(X), np.array(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # ✅ Check if model file exists before training
    if os.path.exists(f"{model_name}.h5"):
        print(f"✅ Loading existing model: {model_name}.h5")
        model = load_model(f"{model_name}.h5", custom_objects={"mse": MeanSquaredError()})  # ✅ Fix applied
    else:
        print(f"🚀 Training new model: {model_name}")
        model = Sequential([
            keras.Input(shape=(30, 1)),
            LSTM(50, activation='relu', return_sequences=True),
            Dropout(0.2),
            LSTM(50, activation='relu'),
            Dropout(0.2),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss=MeanSquaredError())  # ✅ Use explicit function
        model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))
        model.save(f"{model_name}.h5")  # ✅ Save model after training

    return model, scaler, X_test, y_test  # ✅ Returns X_test and y_test

# Train or Load Models
solar_model, solar_scaler, X_test_solar, y_test_solar = train_lstm_model(solar_data, "ALLSKY_SFC_SW_DWN", "solar_model")
wind_model, wind_scaler, X_test_wind, y_test_wind = train_lstm_model(wind_data, "WS10M", "wind_model")


✅ Loading existing model: solar_model.h5
✅ Loading existing model: wind_model.h5


In [ ]:
def forecast_next_year(model, scaler, last_30_days):
    """Generates forecasts for the next year using only numerical values."""
    # ✅ Remove non-numeric columns (like 'Plant' names)
    last_30_days = last_30_days[:, 0].reshape(-1, 1)  # Keep only the feature column

    # ✅ Scale the last 30 days data
    future_inputs = list(scaler.transform(last_30_days))
    predictions = []

    for _ in range(365):  # Predict for the next 365 days
        future_array = np.array(future_inputs[-30:]).reshape(1, 30, 1)
        pred = model.predict(future_array)[0][0]
        predictions.append(pred)
        future_inputs.append([pred])  # Add prediction to future inputs

    return scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()


In [ ]:
# ✅ Select only numerical columns, drop 'Plant' names
last_30_solar = solar_data.drop(columns=["Plant"]).iloc[-30:].values
last_30_wind = wind_data.drop(columns=["Plant"]).iloc[-30:].values

# ✅ Generate forecasts
solar_forecast = forecast_next_year(solar_model, solar_scaler, last_30_solar)
wind_forecast = forecast_next_year(wind_model, wind_scaler, last_30_wind)

# ✅ Print sample forecast values
print("Solar Forecast (First 10 days):", solar_forecast[:10])
print("Wind Forecast (First 10 days):", wind_forecast[:10])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 470ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def evaluate_model(model, X_test, y_test, scaler):
    """Evaluates model accuracy using MAE, MSE, and RMSE."""
    y_pred = model.predict(X_test, verbose=0)  # ✅ Suppress logs
    y_pred = scaler.inverse_transform(y_pred)  # ✅ Convert back to original scale
    y_test = scaler.inverse_transform(y_test)  # ✅ Convert actual values

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    print(f"📊 Model Accuracy Metrics:")
    print(f"✅ Mean Absolute Error (MAE): {mae:.4f}")
    print(f"✅ Mean Squared Error (MSE): {mse:.4f}")
    print(f"✅ Root Mean Squared Error (RMSE): {rmse:.4f}")

    return {"MAE": mae, "MSE": mse, "RMSE": rmse}


In [ ]:
print("🌞 Solar Model Accuracy:")
solar_metrics = evaluate_model(solar_model, X_test_solar, y_test_solar, solar_scaler)

print("\n💨 Wind Model Accuracy:")
wind_metrics = evaluate_model(wind_model, X_test_wind, y_test_wind, wind_scaler)


🌞 Solar Model Accuracy:
📊 Model Accuracy Metrics:
✅ Mean Absolute Error (MAE): 1.6924
✅ Mean Squared Error (MSE): 463.6409
✅ Root Mean Squared Error (RMSE): 21.5323

💨 Wind Model Accuracy:
📊 Model Accuracy Metrics:
✅ Mean Absolute Error (MAE): 0.7516
✅ Mean Squared Error (MSE): 0.9983
✅ Root Mean Squared Error (RMSE): 0.9991


In [ ]:
# Scale both input (X) and output (y)
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test_scaled = scaler_X.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1))

# Train the model on scaled data
model.fit(X_train_scaled, y_train_scaled, epochs=50, batch_size=16, validation_data=(X_test_scaled, y_test_scaled))


NameError: name 'X_train' is not defined

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

def preprocess_data(data, feature):
    """Scales both input features (X) and target values (y), and splits into train & test sets."""
    data = data[[feature]].dropna().values  # Keep only the feature column

    # Define scalers
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()

    # Scale input data
    data_scaled = scaler_X.fit_transform(data)

    # Create sequences for LSTM
    X, y = [], []
    for i in range(30, len(data_scaled) - 1):
        X.append(data_scaled[i-30:i])  # Past 30 days as input
        y.append(data_scaled[i])  # Target is the next day's value

    X, y = np.array(X), np.array(y)

    # ✅ Split into train & test sets **BEFORE** scaling y
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # ✅ Scale target values (`y_train`, `y_test`)
    y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1))
    y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1))

    return X_train, X_test, y_train_scaled, y_test_scaled, scaler_X, scaler_y

# ✅ Preprocess Solar & Wind Data
X_train_solar, X_test_solar, y_train_solar, y_test_solar, scaler_X_solar, scaler_y_solar = preprocess_data(solar_data, "ALLSKY_SFC_SW_DWN")
X_train_wind, X_test_wind, y_train_wind, y_test_wind, scaler_X_wind, scaler_y_wind = preprocess_data(wind_data, "WS10M")


In [ ]:
solar_data = prepare_data_for_training(ntpc_solar_plants, "ALLSKY_SFC_SW_DWN")
wind_data = prepare_data_for_training(ntpc_wind_plants, "WS10M")


Fetching data for Bhadla Solar ((27.485071, 71.997756))...
Fetching data for Jetsar Solar ((9.0128, 76.9144))...
Fetching data for Bhensada Solar ((24.1, 78.8))...
Fetching data for NTPC Hybrid SECI V ((23.2064, 78.382))...
Fetching data for Dayapar Wind Farm ((23.6348, 68.9115))...


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras import Input
from tensorflow.keras.optimizers import Adamax

def build_improved_lstm():
    """Builds an improved LSTM model with correct input layer definition."""
    model = Sequential([
        Input(shape=(30, 1)),  # ✅ Correct way to define input shape
        LSTM(100, activation='tanh', return_sequences=True),
        Dropout(0.2),
        LSTM(100, activation='tanh'),
        Dropout(0.2),
        Dense(1)
    ])

    model.compile(optimizer=Adamax(), loss="mse")
    return model

# ✅ Load or fine-tune models using this updated architecture
solar_model = build_improved_lstm()
wind_model = build_improved_lstm()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# ✅ Use Early Stopping to avoid overfitting
early_stop = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

# ✅ Fine-tune Solar Model (Train for 10-20 More Epochs)
if solar_model:
    print("🌞 Fine-tuning Solar Model...")
    solar_model.fit(X_train_solar, y_train_solar, epochs=20, batch_size=16, validation_data=(X_test_solar, y_test_solar), callbacks=[early_stop])
    solar_model.save("solar_model.h5")  # ✅ Save updated model

# ✅ Fine-tune Wind Model
if wind_model:
    print("\n💨 Fine-tuning Wind Model...")
    wind_model.fit(X_train_wind, y_train_wind, epochs=20, batch_size=16, validation_data=(X_test_wind, y_test_wind), callbacks=[early_stop])
    wind_model.save("wind_model.h5")  # ✅ Save updated model


🌞 Fine-tuning Solar Model...
Epoch 1/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 27s 41ms/step - loss: 0.0353 - val_loss: 5.5145e-04
Epoch 2/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 39s 38ms/step - loss: 0.0038 - val_loss: 5.5576e-04
Epoch 3/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 43s 42ms/step - loss: 0.0034 - val_loss: 4.6656e-04
Epoch 4/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 21s 38ms/step - loss: 0.0033 - val_loss: 7.8599e-04
Epoch 5/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 22s 40ms/step - loss: 0.0030 - val_loss: 5.0489e-04
Epoch 6/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 40s 38ms/step - loss: 0.0028 - val_loss: 5.2344e-04
Epoch 7/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 43s 41ms/step - loss: 0.0025 - val_loss: 4.6621e-04
Epoch 8/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 39s 38ms/step - loss: 0.0026 - val_loss: 4.8771e-04
Epoch 9/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 41s 39ms/step - loss: 0.0023 - val_loss: 5.2206e-04
Epoch 10/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 41s 39ms/step - loss: 0.0021 - val_loss: 5.0954e-04
Epoch 11/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 22s 40ms/


💨 Fine-tuning Wind Model...
Epoch 1/20
364/364 ━━━━━━━━━━━━━━━━━━━━ 18s 40ms/step - loss: 0.0127 - val_loss: 0.0134
Epoch 2/20
364/364 ━━━━━━━━━━━━━━━━━━━━ 14s 38ms/step - loss: 0.0100 - val_loss: 0.0125
Epoch 3/20
364/364 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - loss: 0.0083 - val_loss: 0.0118
Epoch 4/20
364/364 ━━━━━━━━━━━━━━━━━━━━ 20s 37ms/step - loss: 0.0080 - val_loss: 0.0113
Epoch 5/20
364/364 ━━━━━━━━━━━━━━━━━━━━ 20s 37ms/step - loss: 0.0076 - val_loss: 0.0102
Epoch 6/20
364/364 ━━━━━━━━━━━━━━━━━━━━ 14s 38ms/step - loss: 0.0072 - val_loss: 0.0091
Epoch 7/20
364/364 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - loss: 0.0066 - val_loss: 0.0083
Epoch 8/20
364/364 ━━━━━━━━━━━━━━━━━━━━ 20s 37ms/step - loss: 0.0066 - val_loss: 0.0078
Epoch 9/20
364/364 ━━━━━━━━━━━━━━━━━━━━ 21s 38ms/step - loss: 0.0059 - val_loss: 0.0074
Epoch 10/20
364/364 ━━━━━━━━━━━━━━━━━━━━ 14s 37ms/step - loss: 0.0058 - val_loss: 0.0070
Epoch 11/20
364/364 ━━━━━━━━━━━━━━━━━━━━ 14s 38ms/step - loss: 0.0057 - val_loss: 0.0069
E

In [24]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

def evaluate_model(model, X_test, y_test, scaler_y):
    """Evaluates model accuracy using MAE, MSE, RMSE."""
    y_pred = model.predict(X_test, verbose=0)
    y_pred = scaler_y.inverse_transform(y_pred)
    y_test = scaler_y.inverse_transform(y_test)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    print(f"📊 Model Accuracy Metrics:")
    print(f"✅ Mean Absolute Error (MAE): {mae:.4f}")
    print(f"✅ Mean Squared Error (MSE): {mse:.4f}")
    print(f"✅ Root Mean Squared Error (RMSE): {rmse:.4f}")

    return {"MAE": mae, "MSE": mse, "RMSE": rmse}

print("🌞 Solar Model Accuracy:")
evaluate_model(solar_model, X_test_solar, y_test_solar, scaler_y_solar)

print("\n💨 Wind Model Accuracy:")
evaluate_model(wind_model, X_test_wind, y_test_wind, scaler_y_wind)


🌞 Solar Model Accuracy:
📊 Model Accuracy Metrics:
✅ Mean Absolute Error (MAE): 0.0017
✅ Mean Squared Error (MSE): 0.0005
✅ Root Mean Squared Error (RMSE): 0.0214

💨 Wind Model Accuracy:
📊 Model Accuracy Metrics:
✅ Mean Absolute Error (MAE): 0.0534
✅ Mean Squared Error (MSE): 0.0051
✅ Root Mean Squared Error (RMSE): 0.0715


{'MAE': 0.053425921467675906,
 'MSE': 0.005105820968510207,
 'RMSE': 0.07145502759435621}

In [25]:
!pip install fastapi uvicorn nest-asyncio pydantic tensorflow pandas numpy scikit-learn


In [27]:
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.models import load_model
import os

def load_trained_model(model_name):
    """Loads an existing trained LSTM model and registers 'mse' as a valid loss function."""
    if os.path.exists(f"{model_name}.h5"):
        print(f"✅ Loading model: {model_name}.h5")
        model = load_model(f"{model_name}.h5", custom_objects={"mse": MeanSquaredError()})  # ✅ Fix applied
        return model
    else:
        print(f"⚠️ Model {model_name}.h5 not found. Train the model first.")
        return None  # Return None if model is missing

# ✅ Load the trained models
solar_model = load_trained_model("solar_model")
wind_model = load_trained_model("wind_model")


✅ Loading model: solar_model.h5
✅ Loading model: wind_model.h5


In [28]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

# ✅ Define API input structure
class ForecastRequest(BaseModel):
    plant_type: str
    last_30_days: list

# ✅ API health check
@app.get("/")
def home():
    return {"message": "Energy Forecasting API is running!"}

# ✅ Define prediction function
def forecast_next_year(model, last_30_days):
    """Generates forecast for the next year based on last 30 days of data."""
    last_30_days = np.array(last_30_days).reshape(-1, 1)
    predictions = []

    for _ in range(365):
        future_array = np.array(last_30_days[-30:]).reshape(1, 30, 1)
        pred = model.predict(future_array, verbose=0)[0][0]
        predictions.append(pred)
        last_30_days = np.append(last_30_days, pred).reshape(-1, 1)

    return predictions

# ✅ API endpoint to get predictions
@app.post("/predict")
def predict_energy(request: ForecastRequest):
    if request.plant_type == "solar" and solar_model:
        forecast = forecast_next_year(solar_model, request.last_30_days)
    elif request.plant_type == "wind" and wind_model:
        forecast = forecast_next_year(wind_model, request.last_30_days)
    else:
        return {"error": "Invalid plant type or model not found. Use 'solar' or 'wind'."}

    return {"forecast": forecast}


In [29]:
from threading import Thread

# ✅ Run FastAPI server in the background
def run_api():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# ✅ Start the API
nest_asyncio.apply()
api_thread = Thread(target=run_api)
api_thread.start()


In [30]:
import requests

# Example: Last 30 days of solar energy data (Replace with real values)
last_30_days_data = [[1.2], [1.3], [1.4], [1.5], [1.6], [1.7], [1.8], [1.9], [2.0], [2.1],
                      [2.2], [2.3], [2.4], [2.5], [2.6], [2.7], [2.8], [2.9], [3.0], [3.1],
                      [3.2], [3.3], [3.4], [3.5], [3.6], [3.7], [3.8], [3.9], [4.0], [4.1]]

response = requests.post("http://0.0.0.0:8000/predict", json={"plant_type": "solar", "last_30_days": last_30_days_data})
print(response.json())


INFO:     127.0.0.1:46210 - "POST /predict HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/fastapi/encoders.py", line 324, in jsonable_encoder
    data = dict(obj)
           ^^^^^^^^^
TypeError: 'numpy.float32' object is not iterable

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/fastapi/encoders.py", line 329, in jsonable_encoder
    data = vars(obj)
           ^^^^^^^^^
TypeError: vars() argument must have __dict__ attribute

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [31]:
def forecast_next_month(model, last_30_days):
    """Generates a 30-day energy forecast based on the last 30 days."""
    last_30_days = np.array(last_30_days).reshape(-1, 1)  # Convert to NumPy array
    predictions = []

    for _ in range(30):  # Predict for the next 30 days instead of 365
        future_array = np.array(last_30_days[-30:]).reshape(1, 30, 1)
        pred = model.predict(future_array, verbose=0)[0][0]  # Get prediction
        predictions.append(float(pred))  # ✅ Convert NumPy float32 → Python float
        last_30_days = np.append(last_30_days, pred).reshape(-1, 1)  # Append prediction

    return predictions  # ✅ Now returns only 30 days of forecast


In [32]:
def forecast_next_month(model, last_30_days):
    """Generates a 30-day energy forecast based on the last 30 days."""
    last_30_days = np.array(last_30_days).reshape(-1, 1)  # Convert to NumPy array
    predictions = []

    print("\n📊 Next 30 Days Forecast:")
    for day in range(1, 31):  # Predict for the next 30 days
        future_array = np.array(last_30_days[-30:]).reshape(1, 30, 1)
        pred = model.predict(future_array, verbose=0)[0][0]  # Get prediction
        predictions.append(float(pred))  # Convert NumPy float32 → Python float
        last_30_days = np.append(last_30_days, pred).reshape(-1, 1)  # Append prediction

        print(f"📅 Day {day}: {pred:.4f} MWh")  # ✅ Print each day's prediction

    return predictions  # ✅ Returns only 30 days of forecast


In [33]:
# Example: Last 30 days of solar energy data (Replace with real values)
last_30_days_data = [[1.2], [1.3], [1.4], [1.5], [1.6], [1.7], [1.8], [1.9], [2.0], [2.1],
                      [2.2], [2.3], [2.4], [2.5], [2.6], [2.7], [2.8], [2.9], [3.0], [3.1],
                      [3.2], [3.3], [3.4], [3.5], [3.6], [3.7], [3.8], [3.9], [4.0], [4.1]]

# Load trained model
solar_model = load_trained_model("solar_model")

# Run forecast and print output
forecast = forecast_next_month(solar_model, last_30_days_data)
print("\n✅ Final 30-Day Forecast:", forecast)


✅ Loading model: solar_model.h5

📊 Next 30 Days Forecast:
📅 Day 1: 1.0343 MWh
📅 Day 2: 1.0495 MWh
📅 Day 3: 1.0506 MWh
📅 Day 4: 1.0469 MWh
📅 Day 5: 1.0406 MWh
📅 Day 6: 1.0336 MWh
📅 Day 7: 1.0269 MWh
📅 Day 8: 1.0209 MWh
📅 Day 9: 1.0159 MWh
📅 Day 10: 1.0118 MWh
📅 Day 11: 1.0085 MWh
📅 Day 12: 1.0059 MWh
📅 Day 13: 1.0038 MWh
📅 Day 14: 1.0023 MWh
📅 Day 15: 1.0010 MWh
📅 Day 16: 1.0001 MWh
📅 Day 17: 0.9994 MWh
📅 Day 18: 0.9989 MWh
📅 Day 19: 0.9985 MWh
📅 Day 20: 0.9982 MWh
📅 Day 21: 0.9980 MWh
📅 Day 22: 0.9978 MWh
📅 Day 23: 0.9977 MWh
📅 Day 24: 0.9976 MWh
📅 Day 25: 0.9975 MWh
📅 Day 26: 0.9975 MWh
📅 Day 27: 0.9974 MWh
📅 Day 28: 0.9974 MWh
📅 Day 29: 0.9974 MWh
📅 Day 30: 0.9974 MWh

✅ Final 30-Day Forecast: [1.0343189239501953, 1.049452781677246, 1.0506435632705688, 1.0469387769699097, 1.040597915649414, 1.033583164215088, 1.0268676280975342, 1.0209194421768188, 1.015892505645752, 1.0117740631103516, 1.0084712505340576, 1.005864143371582, 1.0038292407989502, 1.0022555589675903, 1.001047134399414, 

In [34]:
!git clone https://github.com/Ankur0191/model3.git

Cloning into 'model3'...


In [37]:
!mv model3.ipynb model3/


mv: cannot stat 'model3.ipynb': No such file or directory
